In [73]:
!pip install rioxarray  xarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 730.4 kB/s eta 0:00:00 0:00:01
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 106.8 MB/s eta 0:00:0000:0100:01
Using cached packaging-23.2-py3-none-any.whl (53 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 129.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 81.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently 

In [76]:
import os
import yaml
from datetime import datetime,timedelta
from yaml.loader import SafeLoader
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
from dask.diagnostics import ProgressBar
from datetime import datetime
import numpy as np
from dask import dataframe as dd

In [10]:
twoMonthsDate = (datetime.now()-timedelta(days=60))
year = twoMonthsDate.strftime('%Y')
month = twoMonthsDate.strftime('%m')
ymlPath = 'conf/chirps.yml'

In [67]:
def readYamlConf(path: str):
    with open(path,'r') as conf:
        confArgs = yaml.load(conf, Loader= SafeLoader)
        #print(confArgs)
    return confArgs

def downloadDataFromS3FilteredByYear(s3Path: str
                                     , downloadPath: str
                                     , year:str 
                                     , month : str
                                    ):
    
    s3Command = f'aws s3 cp {s3Path} {downloadPath} --recursive --exclude "*" --include "chirps-v2.0.{year}.{month}.*"'
    
    return os.system(s3Command)

def uncompressGunzip (path:str):
    command = f'cd {path}| gunzip {path}*'
    os.system(command)
    

def VtransformCHIRTSToNc(f):
    allFiles = xr.open_dataset(f'data/{f}')
    dateContent = f.split('.')
    date = datetime(int(dateContent[-4]),int(dateContent[-3]),int(dateContent[-2]))
    filename = date.strftime('%Y%m%d')
    time_da = xr.DataArray({'time': date})
    allFiles = allFiles.assign_coords(time = date)
    allFiles = allFiles.expand_dims(dim="time")
    allFiles = allFiles.rename_vars(name_dict = {'band_data':'tmin'})
    allFiles = allFiles.rename(name_dict   = {'x':'lon','y':'lat'})
    allFiles = allFiles.drop('band')
    allFiles = allFiles.drop('spatial_ref')
    allFiles.to_netcdf(f'data/toZarr/{filename}.nc')
    

In [46]:
confArgs = readYamlConf(ymlPath)
chirpsArgs = confArgs['chirpsArguments']
del confArgs

In [52]:
logCapture = downloadDataFromS3FilteredByYear(chirpsArgs['awsChirpsPath']
                                 , chirpsArgs['pathforTGZ']
                                 ,'2023'
                                 ,'07'
                                )


download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.09.tif.gz to tgz/chirps-v2.0.2023.07.09.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.03.tif.gz to tgz/chirps-v2.0.2023.07.03.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.06.tif.gz to tgz/chirps-v2.0.2023.07.06.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.04.tif.gz to tgz/chirps-v2.0.2023.07.04.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.08.tif.gz to tgz/chirps-v2.0.2023.07.08.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.10.tif.gz to tgz/chirps-v2.0.2023.07.10.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRP

In [68]:
uncompressGunzip(chirpsArgs['pathforTGZ'])

In [69]:
tiffToTransform = os.listdir(chirpsArgs['pathforTGZ'])

In [77]:
tiffToTransformDf = pd.DataFrame()
tiffToTransformDf['files'] = tiffToTransform

In [78]:
tiffToTransformDf

,files
0,chirps-v2.0.2023.07.09.tif
1,chirps-v2.0.2023.07.12.tif
2,chirps-v2.0.2023.07.15.tif
3,chirps-v2.0.2023.07.10.tif
4,chirps-v2.0.2023.07.26.tif
5,chirps-v2.0.2023.07.23.tif
6,chirps-v2.0.2023.07.13.tif
7,chirps-v2.0.2023.07.31.tif
8,chirps-v2.0.2023.07.17.tif
9,chirps-v2.0.2023.07.25.tif
